<a href="https://colab.research.google.com/github/YoheiFukuhara/nlp100-2020/blob/main/09.RNN%2CCNN/88_%E3%83%91%E3%83%A9%E3%83%A1%E3%83%BC%E3%82%BF%E3%83%81%E3%83%A5%E3%83%BC%E3%83%8B%E3%83%B3%E3%82%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

問題85や問題87のコードを改変し，ニューラルネットワークの形状やハイパーパラメータを調整しながら，高性能なカテゴリ分類器を構築せよ．

In [1]:
!pip install keras-tuner
#!pip show keras-tuner
#import keras_tuner as kt

     |████████████████████████████████| 98 kB 3.2 MB/s 


In [2]:
import IPython
import keras_tuner as kt
import numpy as np
import nltk
from gensim.models import KeyedVectors
import pandas as pd
import tensorflow as tf
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!python --version
!pip show keras-tuner numpy gensim google tensorflow nltk pandas

Python 3.7.12
Name: keras-tuner
Version: 1.1.0
Summary: Hypertuner for Keras
Home-page: https://github.com/keras-team/keras-tuner
Author: The KerasTuner authors
Author-email: kerastuner@google.com
License: Apache License 2.0
Location: /usr/local/lib/python3.7/dist-packages
Requires: kt-legacy, ipython, tensorboard, numpy, scipy, requests, packaging
Required-by: 
---
Name: numpy
Version: 1.19.5
Summary: NumPy is the fundamental package for array computing with Python.
Home-page: https://www.numpy.org
Author: Travis E. Oliphant et al.
Author-email: None
License: BSD
Location: /usr/local/lib/python3.7/dist-packages
Requires: 
Required-by: yellowbrick, xgboost, xarray, wordcloud, torchvision, torchtext, tifffile, thinc, Theano-PyMC, tensorflow, tensorflow-probability, tensorflow-hub, tensorflow-datasets, tensorboard, tables, statsmodels, spacy, sklearn-pandas, seaborn, scs, scipy, scikit-learn, scikit-image, resampy, qdldl, PyWavelets, python-louvain, pystan, pysndfile, pymc3, pyerfa, pyem

In [4]:
%%time
BASE_PATH = '/content/drive/MyDrive/ColabNotebooks/ML/NLP100_2020/'
max_len = 0
vocabulary = []
w2v_model = KeyedVectors.load_word2vec_format(BASE_PATH+'07.WordVector/input/GoogleNews-vectors-negative300.bin.gz', binary=True)

CPU times: user 2min 30s, sys: 5.41 s, total: 2min 36s
Wall time: 2min 47s


In [5]:
def read_dataset(type_):
    global max_len
    global vocabulary
    df = pd.read_table(BASE_PATH+'06.MachineLearning/'+type_+'.feature.txt')
    df.info()
    sr_title = df['title'].str.split().explode()
    max_len_ = df['title'].map(lambda x: len(x.split())).max()
    if max_len < max_len_:
        max_len = max_len_
    if len(vocabulary) == 0:
        vocabulary = [k for k, v in nltk.FreqDist(sr_title).items() if v > 1]
    else:
        vocabulary.extend([k for k, v in nltk.FreqDist(sr_title).items() if v > 1])
    y = df['category'].replace({'b':0, 't':1, 'e':2, 'm':3})
    return df['title'], tf.keras.utils.to_categorical(y, dtype='int32')  # 4値分類なので訓練・検証・テスト共通でone-hot化

In [6]:
X_train, y_train = read_dataset('train')
X_valid, y_valid = read_dataset('valid')
X_test, y_test = read_dataset('test') # あまりこだわらずにテストデータセットも追加

# setで重複削除し、タプル形式に設定
tup_voc = tuple(set(vocabulary))

print(f'vocabulary size before removing duplicates: {len(vocabulary)}')
print(f'vocabulary size after removing duplicates: {len(tup_voc)}')
print(f'sample vocabulary: {tup_voc[:10]}')
print(f'max length is {max_len}')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10684 entries, 0 to 10683
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   title     10684 non-null  object
 1   category  10684 non-null  object
dtypes: object(2)
memory usage: 167.1+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1336 entries, 0 to 1335
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   title     1336 non-null   object
 1   category  1336 non-null   object
dtypes: object(2)
memory usage: 21.0+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1336 entries, 0 to 1335
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   title     1336 non-null   object
 1   category  1336 non-null   object
dtypes: object(2)
memory usage: 21.0+ KB
vocabulary size before removing duplicates: 11089
vocabulary size after removing duplicates: 7802
sa

In [7]:
vectorize_layer = tf.keras.layers.TextVectorization(
 output_mode='int',
 vocabulary=tup_voc,
 output_sequence_length=max_len)

print(f'vocabulary size is {vectorize_layer.vocabulary_size()}')

vocabulary size is 7804


In [8]:
%%time
embedding_dim = 300
hits = 0
misses = 0

embedding_matrix = np.zeros((vectorize_layer.vocabulary_size(), embedding_dim))
for i, word in enumerate(vectorize_layer.get_vocabulary()):
    try:
        embedding_matrix[i] = w2v_model.get_vector(word)
        hits += 1
        # Words not found in embedding index will be all-zeros.
        # This includes the representation for "padding" and "OOV"
    except:
        misses += 1
        if misses < 7:  # Show 6 words as example
            print(word)


[UNK]
knowle
S&P
Mccurdy
ftse
CPU times: user 79.8 ms, sys: 13 ms, total: 92.8 ms
Wall time: 144 ms


In [9]:
embedding_layer = tf.keras.layers.Embedding(
    vectorize_layer.vocabulary_size(),
    embedding_dim,
    embeddings_initializer=tf.keras.initializers.Constant(embedding_matrix),
    trainable=False
)

In [10]:
def conv_layer(kernel_token, in2emb, filter):
    conv = tf.keras.layers.Conv1D(filters=filter, 
                                 kernel_size=embedding_dim*kernel_token, 
                                 padding='same',
                                 strides=embedding_dim,
                                 activation='relu',
                                 name='Conv1D_'+str(kernel_token))(in2emb)
    conv = tf.keras.layers.MaxPooling1D(pool_size=2, 
                                        name='MaxPooling1D_'+str(kernel_token))(conv)
    conv = tf.keras.layers.Flatten(name='Flatten_'+str(kernel_token))(conv)
    return conv

In [ ]:
def model_builder(hp):
    hp_gru_units = hp.Int('units', min_value = 32, max_value = 96, step = 32)
    hp_gru_dout  = hp.Choice('gru_dropout', values = [0.2, 0.3, 0.4]) 
    hp_filter = hp.Int('filter', min_value = 2, max_value = 4, step = 1)
    hp_dout  = hp.Choice('dropout', values = [0.2, 0.3, 0.4]) 

    input = tf.keras.Input(shape=(1,), dtype=tf.string)
    in2emb = vectorize_layer(input)
    in2emb = embedding_layer(in2emb)
    rnn = tf.keras.layers.Bidirectional(
        tf.keras.layers.GRU(hp_gru_units,
                            dropout=hp_gru_dout,
                            return_sequences=True))(in2emb)
    rnn = tf.keras.layers.Bidirectional(
        tf.keras.layers.GRU(hp_gru_units,
                            dropout=hp_gru_dout))(rnn)

    in2emb = tf.keras.layers.Reshape((max_len * embedding_dim, 1))(in2emb)

    conv3 = conv_layer(2, in2emb, hp_filter)
    conv2 = conv_layer(3, in2emb, hp_filter)
    conv4 = conv_layer(4, in2emb, hp_filter)

    last = tf.keras.layers.concatenate([conv2, conv3, conv4, rnn])
    last = tf.keras.layers.Dropout(hp_dout)(last)
    last = tf.keras.layers.Dense(4, activation='softmax')(last)

    model = tf.keras.models.Model(inputs=input, outputs=last)
    model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['acc'])
    return model

In [11]:
tuner = kt.Hyperband(model_builder,
                     objective = 'val_loss', 
                     max_epochs = 30,
                     factor = 3,
                     directory = './',
                     project_name = 'nlp100-2020')

In [12]:
class ClearTrainingOutput(tf.keras.callbacks.Callback):
  def on_train_end(*args, **kwargs):
    IPython.display.clear_output(wait = True)

In [ ]:
%%time
tuner.search(X_train, y_train, epochs=100, 
             validation_data=(X_valid, y_valid), 
             callbacks = [ClearTrainingOutput()])

# Get the optimal hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials = 1)[0]

print(best_hps.values)

Trial 56 Complete [00h 01m 03s]
val_loss: 0.7480563521385193

Best val_loss So Far: 0.4708493947982788
Total elapsed time: 00h 56m 07s

Search: Running Trial #57

Hyperparameter    |Value             |Best Value So Far 
units             |64                |96                
gru_dropout       |0.3               |0.2               
filter            |3                 |4                 
dropout           |0.4               |0.2               
tuner/epochs      |4                 |30                
tuner/initial_e...|0                 |10                
tuner/bracket     |2                 |3                 
tuner/round       |0                 |3                 

Epoch 1/4
334/334 [==============================] - 23s 45ms/step - loss: 1.1717 - acc: 0.4955 - val_loss: 1.0956 - val_acc: 0.6946
Epoch 2/4
334/334 [==============================] - 14s 41ms/step - loss: 1.0309 - acc: 0.6802 - val_loss: 0.9404 - val_acc: 0.7507
Epoch 3/4
334/334 [==============================] - 14s 

In [ ]:
tuner.search_space_summary()
tuner.results_summary()

In [ ]:
model = tuner.hypermodel.build(best_hps)
model.summary()
tf.keras.utils.plot_model(model, show_shapes=True)

In [ ]:
model.fit(X_train, y_train, epochs=100, validation_data=(X_valid, y_valid),
          callbacks=[tf.keras.callbacks.EarlyStopping(patience=5, restore_best_weights=False)])

In [ ]:
model.evaluate(X_test, y_test)